In [ ]:
#!/usr/bin/env


"""


split, clean, and self-cal continuum and line data
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

LupusIII_1003  15:47:56.940 -35:14:34.800
Class II, K7


In [ ]:
field = 20                                     # CHANGEME 
file_ms = '../science_calibrated.ms'
contspw   = '2,3,4'
contspw_w = [128,3840,1920] 
robust     = 0.5
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
refant     = 'DA52'                            # CHANGEME
xc         = 326                               # CHANGEME
yc         = 308                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 0.5                               # CHANGEME
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Split Off Continuum 

split off field from full ms


In [ ]:
split(vis = file_ms,
      outputvis = 'f'+str(field)+'.vis',
      field = field,
      datacolumn = 'data')


split off continuum (take the large bw spw and average


In [ ]:
split(vis = 'f'+str(field)+'.vis',
      outputvis = 'f'+str(field)+'_cont.vis',
      spw = contspw,
      width = contspw_w,
      datacolumn = 'data')


plot uv-distance vs. amplitude


In [ ]:
plotms(vis='f'+str(field)+'_cont.vis',
       xaxis='uvdist',yaxis='amp',
       coloraxis='spw')
       # plotfile='f'+str(field)+'_ampuv_orig.png'
       # showgui=False,
       # highres=True,
       # overwrite=True)


source is unresolved
find antenna close to center of configuration
check pipeline log that this ant is OK


In [ ]:
plotants(vis='f'+str(field)+'_cont.vis') #, figfile='f'+str(field)+'_ants.png')


#  Clean continuum before selfcal 

light clean (100 iterations) around BOTH sources
os.system('rm -rf f'+str(field)+'_cont_b4sc*')


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_b4sc',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold = '0.0mJy',
      interactive = True,
      mask = '',
      cell = cell,
      imsize = imsize,
      weighting = 'briggs',
      robust = robust,
      imagermode = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_b4sc.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_b4sc.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 19.83 mJy, rms = 0.44 mJy, S/N = 44.7


#  Self-Calibrate 1 

first combine all the data by time (solint = inf)
i.e., phase self-cal over entire integration time


In [ ]:
gaincal(vis = 'f'+str(field)+'_cont.vis',
        caltable = 'f'+str(field)+'_cont_pcal1',
        refant = refant,
        solint = 'inf',
        combine = 'spw',
        gaintype = 'T',
        spw = '',
        calmode = 'p',
        minblperant = 4,
        minsnr = 3)


plot phase for each antenna


In [ ]:
plotcal(caltable = 'f'+str(field)+'_cont_pcal1',
        xaxis = 'time',
        yaxis = 'phase',
        spw = '',
        iteration = 'antenna',
        subplot = 421,
        plotrange = [0,0,-200,200]) # narrow yrange phases close to zero


no variation between integration times (expected since solin=inf)
note DV02 has no data (100% flagged in pipeline calibration)
apply calibration to data


In [ ]:
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal1'],
        spwmap = [0,0,0],
        calwt = T,
        flagbackup = F)


clean self-calibrated data


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_pcal1_clean',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold   = '0.0mJy',
      interactive = False,
      mask = 'f'+str(field)+'_cont_b4sc.mask',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = robust,
      imagermode  = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_pcal1_clean.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_pcal1_clean.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)
Peak = 22.06 mJy, rms = 0.39 mJy, S/N = 56.9  (slightly better)


inspect images


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_pcal1_clean.image'}])


calibration looks better


#  Self-Calibrate 2 

decrease phase self-cal solution interval to a few times integration time
int = 6s (from X125.log)


In [ ]:
gaincal(vis = 'f'+str(field)+'_cont.vis',
        caltable = 'f'+str(field)+'_cont_pcal2',
        refant = refant,
        solint = '20s',                           # CHANGEME
        combine = 'spw',
        gaintype = 'T',
        spw = '',
        calmode = 'p',
        minblperant = 4,
        minsnr = 3)
plotcal(caltable = 'f'+str(field)+'_cont_pcal2',
        xaxis = 'time',
        yaxis = 'phase',
        spw = '',
        iteration = 'antenna',
        subplot = 421,
        plotrange = [0,0,-200,200])
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal2'],
        spwmap = [0,0,0],
        calwt = T,
        flagbackup = F)
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_pcal2_clean',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold   = '0.0mJy',
      interactive = False,
      mask = 'f'+str(field)+'_cont_b4sc.mask',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = robust,
      imagermode  = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_pcal2_clean.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_pcal2_clean.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 22.04 mJy, rms = 0.39 mJy, S/N = 56.8 (same as pcal1)
inspection of the image shows no change from pcal1


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_pcal1_clean.image'},
               {'file':'f'+str(field)+'_cont_pcal2_clean.image'}])


no change between pcal 1 and 2, so will use pcal1


#  Best Continuum Map 

so now run the same applycal but with flagbackup = T,


In [ ]:
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal1'],         # CHANGEME
        spwmap = [0,0,0],
        calwt = T,
        flagbackup = T)


deep clean, trying different robust weights
os.system('rm -rf f'+str(field)+'_cont_best*')


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_best',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 2000,
      threshold   = '0.0mJy',
      interactive = True,
      mask = '',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = 0.5,                                      # CHANGEME
      imagermode  = imagermode)


placed mask around otuer continuum contour 
stopped after 200 iterations once the inside became green


In [ ]:
im_max = imstat(imagename = 'f'+str(field)+'_cont_best.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
bmaj = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beammajor")
bmin = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beamminor")
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)
print 'Beam = {0:.2f} x {1:.2f} arcsec'.format(bmaj.get('value'),bmin.get('value'))


robust = 0.5
Peak = 22.17 mJy, rms = 0.39 mJy, S/N = 56.8
Beam = 0.35 x 0.29 arcsec
save this to a fits file


In [ ]:
exportfits(imagename='f'+str(field)+'_cont_best.image', fitsimage='f'+str(field)+'_cont.fits')


compare to before self-cal (worth the trade-off with noise to get more definition of second peak)


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_best.image'}])


measure flux
imview(raster=[{'file':'f'+str(field)+'_cont_best.image'}])


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 31.69 mJy, rms = 0.39 mJy, S/N = 81.2
re-center image on source and use get_flux.py to get COG flux


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cont_cropped.image',
             infile  = 'f'+str(field)+'_cont.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cont_cropped.image',
           fitsimage = 'f'+str(field)+'_cont_cropped.fits')


#  Measure flux with UVMODELFIT 

calculate offset from phase center in arcsec


In [ ]:
pixscale = 0.03             # must match 'cell'                 
dx = pixscale*(320.0-xc)    # offset to east (left)
dy = pixscale*(yc-320.0)    # offset to north (up)


measure flux as gaussian


In [ ]:
uvmodelfit(vis       = 'f'+str(field)+'_cont.vis',
           comptype  = 'G',
           sourcepar = [im_flux,dx,dy,0.5,0.7,-90.0],
           varypar   = [T,T,T,T,T,T],
           niter     = 10)


In [ ]:
reduced chi2=1.39927
I = 0.0304255 +/- 0.000552454
x = -0.193787 +/- 0.00285264 arcsec
y = -0.359775 +/- 0.00219143 arcsec
a = 0.24507 +/- 0.0100039 arcsec
r = 0.671193 +/- 0.0450048
p = -85.7045 +/- 4.71919 deg
consistent with aperture method
15:47:56.924 -35:14:35.160
